In [2]:
# from langchain_experimental.sql import SQLDatabaseChain
from langchain_community.utilities.sql_database import SQLDatabase
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData
from sqlalchemy.sql import text

In [ ]:


engine = create_engine('sqlite:///participants.db')
metadata = MetaData()

# Define the participants table
participants_table = Table(
    'participants', metadata,
    Column('sp_id', Integer, primary_key=True),
    Column('age', Integer),
    Column('gender', String),
    Column('years_of_experience', Integer),
    Column('skills', String),
    Column('education_specialization', String),
    Column('past_jobs', String)
)

# Create the table
metadata.create_all(engine)
def populate_fake_data():
    fake_data = [
        {"sp_id": 1, "age": 35, "gender": "Male", "years_of_experience": 10, "skills": "Backend Development, Python, APIs", "education_specialization": "Computer Science", "past_jobs": "Software Engineer at XYZ"},
        {"sp_id": 2, "age": 28, "gender": "Female", "years_of_experience": 5, "skills": "Frontend Development, React, CSS", "education_specialization": "Information Technology", "past_jobs": "UI Developer at ABC"},
        {"sp_id": 3, "age": 40, "gender": "Male", "years_of_experience": 15, "skills": "DevOps, CI/CD, Kubernetes", "education_specialization": "Systems Engineering", "past_jobs": "DevOps Engineer at DEF"},
        {"sp_id": 4, "age": 32, "gender": "Female", "years_of_experience": 8, "skills": "Data Science, Machine Learning, R", "education_specialization": "Data Science", "past_jobs": "Data Scientist at GHI"},
        {"sp_id": 5, "age": 45, "gender": "Male", "years_of_experience": 20, "skills": "Database Management, SQL, Oracle", "education_specialization": "Database Administration", "past_jobs": "DBA at JKL"},
        {"sp_id": 6, "age": 29, "gender": "Female", "years_of_experience": 6, "skills": "Mobile Development, Swift, Android", "education_specialization": "Mobile Computing", "past_jobs": "Mobile Developer at MNO"},
        {"sp_id": 7, "age": 38, "gender": "Male", "years_of_experience": 12, "skills": "Cybersecurity, Ethical Hacking, Firewalls", "education_specialization": "Cybersecurity", "past_jobs": "Security Analyst at PQR"},
        {"sp_id": 8, "age": 27, "gender": "Female", "years_of_experience": 4, "skills": "Cloud Computing, AWS, Azure", "education_specialization": "Cloud Technology", "past_jobs": "Cloud Engineer at STU"},
        {"sp_id": 9, "age": 36, "gender": "Male", "years_of_experience": 14, "skills": "AI Research, NLP, TensorFlow", "education_specialization": "Artificial Intelligence", "past_jobs": "AI Specialist at VWX"},
        {"sp_id": 10, "age": 33, "gender": "Female", "years_of_experience": 9, "skills": "Project Management, Agile, Scrum", "education_specialization": "Business Administration", "past_jobs": "Project Manager at YZA"}
    ]

    with engine.begin() as conn:
        conn.execute(participants_table.insert(), fake_data)

populate_fake_data()

# sql_db = SQLDatabase(engine)

sql_query = "SELECT * FROM participants"
sql_query_input = text(sql_query)
with engine.begin() as conn:
    results = conn.execute(sql_query_input).fetchall()

In [1]:
from langchain_community.utilities import SQLDatabase
import argparse
import json
from typing import List, Optional
from pydantic import BaseModel, Field
import pdb
from typing_extensions import Annotated
from typing_extensions import TypedDict


from langchain_core.documents.base import Document
from langchain_experimental.sql import SQLDatabaseChain
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain import hub
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData
from sqlalchemy.sql import text

from dotenv import load_dotenv
load_dotenv()


True

In [2]:
structured_engine = create_engine('sqlite:///participants_structured.db')
unstructured_engine = create_engine('sqlite:///participants_unstructured.db')
structured_metadata = MetaData()
unstructured_metadata = MetaData()

participants_structured_table = Table(
    'participants_structured', structured_metadata,
    Column('sp_id', Integer, primary_key=True),
    Column('age', Integer),
    Column('gender', String),
    Column('years_of_experience', Integer)
)

participants_unstructured_table = Table(
    'participants_unstructured', unstructured_metadata,
    Column('sp_id', Integer, primary_key=True),
    Column('skills', String),
    Column('education_specialization', String),
    Column('past_jobs', String)
)

structured_metadata.create_all(structured_engine)
unstructured_metadata.create_all(unstructured_engine)

def populate_fake_data():
    structured_data = [
        {"sp_id": 1, "age": 35, "gender": "Male", "years_of_experience": 10},
        {"sp_id": 2, "age": 28, "gender": "Female", "years_of_experience": 5},
        {"sp_id": 3, "age": 40, "gender": "Male", "years_of_experience": 15},
        {"sp_id": 4, "age": 32, "gender": "Female", "years_of_experience": 8},
        {"sp_id": 5, "age": 45, "gender": "Male", "years_of_experience": 20},
        {"sp_id": 6, "age": 29, "gender": "Female", "years_of_experience": 6},
        {"sp_id": 7, "age": 38, "gender": "Male", "years_of_experience": 12},
        {"sp_id": 8, "age": 27, "gender": "Female", "years_of_experience": 4},
        {"sp_id": 9, "age": 36, "gender": "Male", "years_of_experience": 14},
        {"sp_id": 10, "age": 33, "gender": "Female", "years_of_experience": 9}
    ]
    unstructured_data = [
        {"sp_id": 1, "skills": "Backend Development, Python, APIs", "education_specialization": "Computer Science", "past_jobs": "Software Engineer at XYZ"},
        {"sp_id": 2, "skills": "Frontend Development, React, CSS", "education_specialization": "Information Technology", "past_jobs": "UI Developer at ABC"},
        {"sp_id": 3, "skills": "DevOps, CI/CD, Kubernetes", "education_specialization": "Systems Engineering", "past_jobs": "DevOps Engineer at DEF"},
        {"sp_id": 4, "skills": "Data Science, Machine Learning, R", "education_specialization": "Data Science", "past_jobs": "Data Scientist at GHI"},
        {"sp_id": 5, "skills": "Database Management, SQL, Oracle", "education_specialization": "Database Administration", "past_jobs": "DBA at JKL"},
        {"sp_id": 6, "skills": "Mobile Development, Swift, Android", "education_specialization": "Mobile Computing", "past_jobs": "Mobile Developer at MNO"},
        {"sp_id": 7, "skills": "Cybersecurity, Ethical Hacking, Firewalls", "education_specialization": "Cybersecurity", "past_jobs": "Security Analyst at PQR"},
        {"sp_id": 8, "skills": "Cloud Computing, AWS, Azure", "education_specialization": "Cloud Technology", "past_jobs": "Cloud Engineer at STU"},
        {"sp_id": 9, "skills": "AI Research, NLP, TensorFlow", "education_specialization": "Artificial Intelligence", "past_jobs": "AI Specialist at VWX"},
        {"sp_id": 10, "skills": "Project Management, Agile, Scrum", "education_specialization": "Business Administration", "past_jobs": "Project Manager at YZA"}
    ]

    with structured_engine.begin() as conn:
        conn.execute(participants_structured_table.insert(), structured_data)
    with unstructured_engine.begin() as conn:
        conn.execute(participants_unstructured_table.insert(), unstructured_data)

populate_fake_data()

# Example query
my_query = text("SELECT * FROM participants_structured")
with structured_engine.begin() as conn:
    results_structured = conn.execute(my_query).fetchall()

my_query = text("SELECT * FROM participants_unstructured")
with unstructured_engine.begin() as conn:
    results_unstructured = conn.execute(my_query).fetchall()

IntegrityError: (sqlite3.IntegrityError) UNIQUE constraint failed: participants_structured.sp_id
[SQL: INSERT INTO participants_structured (sp_id, age, gender, years_of_experience) VALUES (?, ?, ?, ?)]
[parameters: [(1, 35, 'Male', 10), (2, 28, 'Female', 5), (3, 40, 'Male', 15), (4, 32, 'Female', 8), (5, 45, 'Male', 20), (6, 29, 'Female', 6), (7, 38, 'Male', 12), (8, 27, 'Female', 4), (9, 36, 'Male', 14), (10, 33, 'Female', 9)]]
(Background on this error at: https://sqlalche.me/e/20/gkpj)

In [ ]:
from langchain_community.utilities import SQLDatabase
import argparse
import json
from typing import List, Optional
from pydantic import BaseModel, Field
import pdb
from typing_extensions import Annotated
from typing_extensions import TypedDict


from langchain_core.documents.base import Document
from langchain_experimental.sql import SQLDatabaseChain
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain import hub
from langchain_community.tools.sql_database.tool import QuerySQLDatabaseTool

from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from sqlalchemy import create_engine, Table, Column, Integer, String, MetaData
from sqlalchemy.sql import text

from dotenv import load_dotenv
load_dotenv()


In [ ]:
llm = ChatOpenAI(model="gpt-4o-mini")
engine = create_engine('sqlite:///participants.db')
metadata = MetaData()
participants_table = Table(
    'participants', metadata,
    Column('sp_id', Integer, primary_key=True),
    Column('age', Integer),
    Column('gender', String),
    Column('years_of_experience', Integer),
    Column('skills', String),
    Column('education_specialization', String),
    Column('past_jobs', String)
)
metadata.create_all(engine)
participants_data = [
    {"sp_id": 1, "age": 35, "gender": "Male", "years_of_experience": 10, "skills": "Backend Development, Python, APIs", "education_specialization": "Computer Science", "past_jobs": "Software Engineer at XYZ"},
    {"sp_id": 2, "age": 28, "gender": "Female", "years_of_experience": 5, "skills": "Frontend Development, React, CSS", "education_specialization": "Information Technology", "past_jobs": "UI Developer at ABC"},
    {"sp_id": 3, "age": 40, "gender": "Male", "years_of_experience": 15, "skills": "DevOps, CI/CD, Kubernetes", "education_specialization": "Systems Engineering", "past_jobs": "DevOps Engineer at DEF"},
    {"sp_id": 4, "age": 32, "gender": "Female", "years_of_experience": 8, "skills": "Data Science, Machine Learning, R", "education_specialization": "Data Science", "past_jobs": "Data Scientist at GHI"},
    {"sp_id": 5, "age": 45, "gender": "Male", "years_of_experience": 20, "skills": "Database Management, SQL, Oracle", "education_specialization": "Database Administration", "past_jobs": "DBA at JKL"},
    {"sp_id": 6, "age": 29, "gender": "Female", "years_of_experience": 6, "skills": "Mobile Development, Swift, Android", "education_specialization": "Mobile Computing", "past_jobs": "Mobile Developer at MNO"},
    {"sp_id": 7, "age": 38, "gender": "Male", "years_of_experience": 12, "skills": "Cybersecurity, Ethical Hacking, Firewalls", "education_specialization": "Cybersecurity", "past_jobs": "Security Analyst at PQR"},
    {"sp_id": 8, "age": 27, "gender": "Female", "years_of_experience": 4, "skills": "Cloud Computing, AWS, Azure", "education_specialization": "Cloud Technology", "past_jobs": "Cloud Engineer at STU"},
    {"sp_id": 9, "age": 36, "gender": "Male", "years_of_experience": 14, "skills": "AI Research, NLP, TensorFlow", "education_specialization": "Artificial Intelligence", "past_jobs": "AI Specialist at VWX"},
    {"sp_id": 10, "age": 33, "gender": "Female", "years_of_experience": 9, "skills": "Project Management, Agile, Scrum", "education_specialization": "Business Administration", "past_jobs": "Project Manager at YZA"}
]
with engine.begin() as conn:
    conn.execute(participants_table.delete())
    conn.execute(participants_table.insert(), participants_data)
sql_db = SQLDatabase(engine)
sql_chain = SQLDatabaseChain(llm=llm, database=sql_db)

In [ ]:
# response = sql_db.run("SELECT * FROM participants WHERE gender = 'Male'", include_columns=True)
# print(response)

query_prompt_template = hub.pull("langchain-ai/sql-query-system-prompt")

assert len(query_prompt_template.messages) == 1
query_prompt_template.messages[0].pretty_print()


In [ ]:



class SQLGenOutput(TypedDict):
    """Generated SQL query."""

    query: Annotated[str, ..., "Syntactically valid SQL query."]

class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str

def write_query(state: State):
    """Generate SQL query to fetch information."""
    prompt = query_prompt_template.invoke(
        {
            "dialect": sql_db.dialect,
            "top_k": 10,
            "table_info": sql_db.get_table_info(),
            "input": state["question"],
        }
    )
    structured_llm = llm.with_structured_output(SQLGenOutput)
    result = structured_llm.invoke(prompt)
    return {"query": result["query"]}


def execute_query(state: State):
    """Execute SQL query."""
    execute_query_tool = QuerySQLDatabaseTool(db=sql_db)
    return {"result": execute_query_tool.invoke(state["query"])}

In [ ]:
sql_query = write_query({"question": "give me all participants over the age of 30 who could be back end engineers"})
sql_response = execute_query({"query": sql_query["query"]})
print(sql_query)
print(sql_response)



In [4]:
import pandas as pd

In [10]:
data = {
    'Name': ['Alice', 'Bob'],
    'Work Experience/From Date': [[2015, 2018], [2016, 2019]],  # Paired
    'Work Experience/To Date': [[2017, 2021], [2018, 2023]],    # Paired
    'Languages': [['English', 'French'], ['Spanish']],         # Independent
    'Skills': [['Python', 'SQL'], ['Java']],                   # Independent
    'Age': [30, 40]  # Non-list column, remains unchanged
}

df = pd.DataFrame(data)

# ✅ Step 1: Explode paired columns together (keeping them aligned)
paired_columns = ['Work Experience/From Date', 'Work Experience/To Date']
df = df.explode(paired_columns, ignore_index=True)  # Ensures alignment

# ✅ Step 2: Define independently exploded columns
independent_columns = ['Languages', 'Skills']

# ✅ Step 3: Explode each independent column separately
for col in independent_columns:
    df = df.explode(col, ignore_index=True)  # Ensures independent expansion

df

,Name,Work Experience/From Date,Work Experience/To Date,Languages,Skills,Age
0,Alice,2015,2017,English,Python,30
1,Alice,2015,2017,English,SQL,30
2,Alice,2015,2017,French,Python,30
3,Alice,2015,2017,French,SQL,30
4,Alice,2018,2021,English,Python,30
5,Alice,2018,2021,English,SQL,30
6,Alice,2018,2021,French,Python,30
7,Alice,2018,2021,French,SQL,30
8,Bob,2016,2018,Spanish,Java,40
9,Bob,2019,2023,Spanish,Java,40
